# Search Engine with Tools and Agent

In [1]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki.name

'wikipedia'

In [3]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

'arxiv'

In [6]:
tools = [wiki,arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\deepa\\Panda 2024\\GenAI\\Langchain Tools and Agents\\langta_env\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [9]:
embedding  = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\deepa\Panda 2024\GenAI\Langchain Tools and Agents\langta_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap = 200)
documents = splitter.split_documents(docs)
vectordb = FAISS.from_documents(documents=documents,embedding=embedding)
retriever = vectordb.as_retriever()


In [17]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith search","search about any information about langsmith")

In [18]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\deepa\\Panda 2024\\GenAI\\Langchain Tools and Agents\\langta_env\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith search', description='search about any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002B209738E50>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectors

## Now Run all these tools by using agent and llm

In [19]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
print(groq_api_key)
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(model="llama3-8b-8192",api_key=groq_api_key)
llm

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


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B24B68C130>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B24B68CCD0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\deepa\Panda 2024\GenAI\Langchain Tools and Agents\langta_env\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
# Creating agent

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm,tools=tools,prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [26]:
# Agent Executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [27]:
agent_executor.invoke({"input":"tell me about langsmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If 

{'input': 'tell me about langsmith?',
 'output': "Here's a summary of LangSmith based on the output:\n\nLangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith integrates seamlessly with LangChain's open-source frameworks, langchain and langgraph, and can be used as a standalone platform.\n\nSome key features of LangSmith include:\n\n1. Logging traces: LangSmith provides multiple ways to log traces, including using the `traceable()` function.\n2. Evaluation: LangSmith allows developers to run evaluations to measure the performance of their applications.\n3. API key creation: LangSmith requires an API key, which can be created on the Settings page.\n4. Environment setup: LangSmith provides environment variables that need to be set up, such as `LANGCHAIN_TRACING_V2` and `OPENAI_API_KEY`.\n\nLangSmith is designed to he

In [28]:
agent_executor.invoke({"input":"what is machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witBased on the result of the tool call, I can continue the conversation.

Machine learning is a fascinating field that has revolutionized many areas of our lives. It's a subset of artificial intelligence that enables machines to learn from data without being explicitly programmed. This means that machines can improve their performance over time by adjusting their algorithms and parameters based on new data.

Machine learning has many applications, including image recognition, natural language processing, recommender systems, and predictive analytics, to name a few. It's used in industries such as healthcare, finance, marketing, and education,

{'input': 'what is machine learning?',
 'output': "Based on the result of the tool call, I can continue the conversation.\n\nMachine learning is a fascinating field that has revolutionized many areas of our lives. It's a subset of artificial intelligence that enables machines to learn from data without being explicitly programmed. This means that machines can improve their performance over time by adjusting their algorithms and parameters based on new data.\n\nMachine learning has many applications, including image recognition, natural language processing, recommender systems, and predictive analytics, to name a few. It's used in industries such as healthcare, finance, marketing, and education, among others.\n\nWould you like to know more about a specific aspect of machine learning, or is there something else you'd like to explore?"}

In [30]:
agent_executor.invoke({"input":"what is paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `wikipedia` with `{'query': 'transformer'}`


Page: Transformer
Summary: In electrical engineering, a transformer is a passive component that transfers electrical energy from one electrical circuit to another circuit, or multiple circuits. A varying current in any coil of the transformer produce
Invoking: `wikipedia` with `{

{'input': 'what is paper 1706.03762 about?',
 'output': 'The paper "Attention Is All You Need" introduced the transformer architecture, which is a deep learning model that can be used for various natural language processing tasks, such as machine translation and text classification.'}

In [31]:
agent_executor.invoke({"input":"tell me about iit kanpur"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'IIT Kanpur'}`


Page: IIT Kanpur
Summary: Indian Institute of Technology Kanpur (IIT Kanpur) is a public institute of technology located in Kanpur, Uttar Pradesh, India. As an Indian Institute of Technology (IIT), it was declared an Institute of National Importance 
Invoking: `wikipedia` with `{'query': 'IIT Kanpur history'}`


Page: IIT Kanpur
Summary: Indian Institute of Technology Kanpur (IIT Kanpur) is a public institute of technology located in Kanpur, Uttar Pradesh, India. As an Indian Institute of Technology (IIT), it was declared an Institute of National Importance IIT Kanpur was founded in 1959, as one of the first Indian Institutes of Technology, by the government of India. It was initially established as a joint collaboration between the Government of India and the University of Kanpur. The foundation stone of the institute was laid on August 2, 1959, by the then Prime Minister of India, Jawaharl

{'input': 'tell me about iit kanpur',
 'output': 'IIT Kanpur was founded in 1959, as one of the first Indian Institutes of Technology, by the government of India. It was initially established as a joint collaboration between the Government of India and the University of Kanpur. The foundation stone of the institute was laid on August 2, 1959, by the then Prime Minister of India, Jawaharlal Nehru.'}